<a href="https://colab.research.google.com/github/albachiarabellaroba/HW3---Group-16/blob/main/Copia_di_main_Albachiara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Homework 3 - Places of the world**

*Authors: Bellaroba Albachiara, Mazza Ludovica*

To get started, let's import all the libraries we will need.

In [ ]:
from bs4 import BeautifulSoup
import bs4
import requests
import os.path
import lxml
import re
import codecs
import csv
import nltk
import pickle
import heapq
import numpy as np
from datetime import datetime
from tqdm import tqdm
import time
import pandas as pd

In order to work in a faster environment, we link our work to a shared drive folder.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

Mounted at /content/drive/


## 1. Data collection

### 1.1. Get the list of places

At this point we're creating a text file with all the link related to the 7200 places we will analyze.

In [ ]:
# NON runnare

linklist = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'w')

for i in tqdm(range(1, 401)):
    link = f'https://www.atlasobscura.com/places?page={i}&sort=likes_count'
    cont = requests.get(link)
    soup = BeautifulSoup(cont.content, features='lxml')
    for a in soup.find_all('a', class_='content-card content-card-place'):
        linklist.write(a.get('href')+'\n')
        
linklist.close()

100%|██████████| 400/400 [02:05<00:00,  3.18it/s]


Control that we have all the link.

In [ ]:
file = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r')
count=0
for i in file:
  if i != '\n':
    count += 1
file.close
print(count)

7200


### 1.2. Crawl places

In [ ]:
# NON runnare

def download(start, finish):
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for i in tqdm(range(start, finish)):
      cart = '/content/drive/MyDrive/ADM_HW3/Html_page/page_{}'.format(i)
      os.makedirs(cart)
      
      f = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r', encoding='utf8')
      l = f.readlines()[(i-1)*18:(i)*18]
      l = [k.rstrip() for k in l]
      f.close

      m = 1+18*(i-1)

      for link in l:
        link = 'https://www.atlasobscura.com' + link
        html = requests.get(link, headers = header)
        if html.status_code != 200:
          time.sleep(200)
          html = requests.get(link, headers = header)


        name = '{}/{}.html'.format(cart, m)
        text = open(name, 'w', encoding='utf8')
        text.write(html.text)
        text.close
        m=m+1


In [ ]:
# NON runnare

download(1, 401)

100%|██████████| 100/100 [26:07<00:00, 15.67s/it]


### 1.3 Parse downloaded pages

In [ ]:
# NON runnare

placeName=[]
placeTags=[]
numPeopleVisited=[]
numPeopleWant=[]
placeDesc=[]
placeShortDesc=[]
placeNearby=[]
placeAddress=[]
placeAlt=[]
placeLong=[]
placeEditors=[]
placePubDate=[]
placeRelatedLists=[]
placeRelatedPlaces=[]
placeURL=[]

In [ ]:
# NON runnare

 for i in tqdm(range(1,401)):
   files = os.listdir('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i))
   for k in range(((18*(int(i)-1))+1), (18*int(i)+1)):
     file = open('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i)+'/'+str(k)+'.html', 'r', encoding='utf8')
     soup=bs4.BeautifulSoup(file, 'lxml')

     try:
       name=soup.find('h1', {'class':'DDPage__header-title'}).text
       placeName.append(name)
     except:
       placeName.append('')
         
     try:
       tag=soup.find_all('a', {'class':'itemTags__link js-item-tags-link'})
       tag=[i.get('href').split('/categories/')[1] for i in tag]
       placeTags.append(tag)
     except:
       placeTags.append('')
       
     try:
       visitors=soup.find_all('div', {'class':'title-md item-action-count'})[0].text.strip()
       numPeopleVisited.append(visitors)
     except:
       numPeopleVisited.append('')
    
     try:
       want=soup.find_all('div', {'class':'title-md item-action-count'})[1].text.strip()
       numPeopleWant.append(want)
     except:
       numPeopleWant.append('')

     try:
       desc=soup.find('div', {'class':'DDP__body-copy'}).text.strip()
       placeDesc.append(desc)
     except:
       placeDesc.append('')

     try:
       shdesc=soup.find('h3', {'class':'DDPage__header-dek'}).text.strip()
       placeShortDesc.append(shdesc)
     except:
       placeShortDesc.append('')

     try:
       nearby=soup.find_all('div', {'class':'DDPageSiderailRecirc__item-title'})
       nearby=[i.get_text() for i in nearby]
       placeNearby.append(nearby)
     except:
       placeNearby.append('')

     try:
       address=soup.find_all('address', {'class':'DDPageSiderail__address'})[0]
       address=address.find('div').get_text(separator=' ').split('\n')[0]
       placeAddress.append(address)
     except:
       placeAddress.append('')

     try:
       coordinates=soup.find_all('div', {'class':'DDPageSiderail__coordinates js-copy-coordinates'})[0].text.strip()
       p=coordinates.split(', ')
       placeAlt.append(p[0])
       placeLong.append(p[1])
     except:
       placeAlt.append('')
       placeLong.append('')

     try:
       editors=soup.find_all('div', {'class':'ugc-editor-icons'})
       editors=[i.findChildren('a')[0].get('href') for i in editors]
       editors=[k.split('/users/')[1] for k in editors]
       placeEditors.append(editors)
     except:
       placeEditors.append('')

     try:
       date=soup.find('div', {'class':'DDPContributor__name'}).text.strip()
       placePubDate.append(date)
     except:
       placePubDate.append('')

     try:
       rellist=soup.find_all('div', {'data-gtm-template':'DDP Footer Recirc Lists'})[0]
       rellist=[i.text.strip() for i in rellist.find_all('h3')]
       placeRelatedLists.append(rellist)
     except:
       placeRelatedLists.append('')

     try:
       relplaces=soup.find_all('div', {'class':'CardRecircSection__title'})
       relplaces=[i for i in relplaces if i.text == 'Related Places'][0]
       relplaces=relplaces.find_all_next('div', {'class':'Card__action-btns vue-js-been-there-everywhere-place'})
       relplaces=[k.get('data-place-title') for k in relplaces]
       placeRelatedPlaces.append(relplaces)
     except:
       placeRelatedPlaces.append('')

     try:
       url=soup.find('meta', attrs={'property':'og:url'})['content']
       placeURL.append(url)
     except:
       placeURL.append('')

100%|██████████| 400/400 [48:41<00:00,  7.30s/it]


In [ ]:
# NON runnare

features=pd.DataFrame(columns=['placeName', 'placeTags', 'numPeopleVisited', 'numPeopleWant', 'placeDesc', 'placeShortDesc',
                                            'placeNearby', 'placeAddress', 'placeAlt', 'placeLong', 'placeEditors', 'placePubDate', 
                                            'placeRelatedLists', 'placeRelatedPlaces', 'placeURL'], index=[1,7200])

for i in tqdm(range(0, 7200)):
  features.at[i, 'placeName'] = placeName[i]
  features.at[i, 'placeTags'] = placeTags[i]
  features.at[i, 'numPeopleVisited']= numPeopleVisited[i]
  features.at[i, 'numPeopleWant']= numPeopleWant[i]
  features.at[i, 'placeDesc']= placeDesc[i]
  features.at[i, 'placeShortDesc']= placeShortDesc[i]
  features.at[i, 'placeNearby']= placeNearby[i]
  features.at[i, 'placeAddress']= placeAddress[i]
  features.at[i, 'placeAlt']= placeAlt[i]
  features.at[i, 'placeLong']= placeLong[i]
  features.at[i, 'placeEditors']= placeEditors[i]
  features.at[i, 'placePubDate']= placePubDate[i]
  features.at[i, 'placeRelatedLists']= placeRelatedLists[i]
  features.at[i, 'placeRelatedPlaces']= placeRelatedPlaces[i]
  features.at[i, 'placeURL']= placeURL[i]
  

100%|██████████| 7200/7200 [00:23<00:00, 303.84it/s]


In [ ]:
#os.makedirs('/content/drive/MyDrive/ADM_HW3/tsv_files')

# NON runnare

placesfeatures = features.to_csv('/content/drive/MyDrive/ADM_HW3/features.tsv', sep='\t', index=False)

In [ ]:
visual=pd.read_csv('/content/drive/MyDrive/ADM_HW3/features.tsv', sep='\t')
visual = visual.drop(labels=[1], axis=0)
visual.head(10)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2625.0,8208.0,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,['anthonyhogg'],"August 9, 2014","[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...
2,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834.0,8606.0,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['rebekah-otto', 'annetta-black']","May 8, 2010",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...
3,Leadenhall Market,"['markets', 'film-locations', 'film']",3142.0,7595.0,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['meghanneal', 'gavin-292130']","August 1, 2016","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...
4,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...",2430.0,7425.0,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'bylow1md']","November 21, 2008","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",http://www.atlasobscura.com/places/wave-organ
5,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...",4456.0,7057.0,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['cpilgrim', 'akazem']","February 13, 2009","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",http://www.atlasobscura.com/places/catacombes-...
6,The Evolution Store,"['wonder-cabinets', 'purveyors-of-curiosities'...",2198.0,6951.0,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","687 Broadway New York, New York, 10012 United ...",40.7281,-73.9948,"['mbison', 'ramosju']","November 21, 2008","[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",http://www.atlasobscura.com/places/evolution-n...
7,Magowan's Infinite Mirror Maze,"['optical-illusions', 'labyrinths', 'optical-o...",909.0,6507.0,"People stumble about the labyrinth, arms outst...",A psychedelic labyrinth on the San Francisco bay.,"['Musée Mécanique', 'Jack Early Park', ""Fior D...","Pier 39, Building O-11 San Francisco, Califor...",37.8100,-122.4104,"['guttersniperebecca', 'douglasthayer']","December 20, 2016","['24 Mesmerizing Mazes and Labyrinths', ""The W...","['The Winchester Mizmaze', 'Cathedral of Thorn...",http://www.atlasobscura.com/places/magowans-in...
8,Underground Tunnels of Los Angeles,"['tunnels', 'subterranean-sites', 'architectur...",660.0,6440.0,While the rest of the country was forced to go...,"During prohibition, corrupt city officials ran...","['Cathedral of Our Lady of the Angels', 'Trifo.

## 2. Search Engine

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Let's define a function that allows us to clean our descriptions

In [ ]:
def clean(line):
  #split into token
  tokens = word_tokenize(line)
  #convert to lowercase
  tokens = [w.lower() for w in tokens]
  #remove punctuation
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic or numeric (e.g. date or years can be usefull)
  words = [word for word in stripped if word.isalnum()]
  #filterout stopwords
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  #stemming
  ps = PorterStemmer()
  stem_sentence=[]

  for word in words:
    stem_sentence.append(ps.stem(word))

  return stem_sentence

Now to make things easier let's isolate the column we want to work on: DESCRIPTIONS

In [ ]:
#questa cosa di desc col ci serve effettivamente? perchè ora usiamo visual per pulire e fare il vocabulary
desc_col=pd.read_csv('/content/drive/MyDrive/ADM_HW3/features.tsv', usecols=['placeDesc'] ,sep='\t')

In [ ]:
def vocabulary(visual):
    dic = {}
    contatore = 1
    for desc in tqdm(visual["placeDesc"]):
        #Cleaning the data
        desc = clean(str(desc))
        for word in desc:
            if word not in dic.keys():
              dic[word]=contatore
              contatore+=1
    return dic

In [ ]:
vocabulary = vocabulary(visual)

100%|██████████| 7200/7200 [00:55<00:00, 129.78it/s]


In [ ]:
# NON runnare
with open('/content/drive/MyDrive/ADM_HW3/vocabulary.pkl', 'wb') as f:
  pickle.dump(vocabulary, f)

In [ ]:
# NON runnare
del vocabulary

In [ ]:
with open('/content/drive/MyDrive/ADM_HW3/vocabulary.pkl', 'rb') as f:
  vocabulary=pickle.load(f)

In [ ]:
print('In our vocabulary there are '+str(len(vocabulary))+' words.')

In our vocabulary there are 48986 words.


In [ ]:
print(vocabulary)

{'open': 1, '1839': 2, 'highgat': 3, 'one': 4, 'london': 5, 'infam': 6, 'cemeteri': 7, 'origin': 8, 'citi': 9, 'magnific': 10, 'seven': 11, 'park': 12, 'initi': 13, 'design': 14, 'carri': 15, 'architect': 16, 'stephen': 17, 'geari': 18, 'among': 19, 'impress': 20, 'victorian': 21, 'egyptian': 22, 'influenc': 23, 'tomb': 24, 'graveston': 25, 'karl': 26, 'marx': 27, 'recogniz': 28, 'glower': 29, 'beard': 30, 'bust': 31, 'scifi': 32, 'author': 33, 'dougla': 34, 'adam': 35, 'jame': 36, 'holman': 37, 'sightless': 38, '19thcenturi': 39, 'adventur': 40, 'known': 41, 'blind': 42, 'travel': 43, 'pioneer': 44, 'human': 45, 'echoloc': 46, 'abl': 47, 'sens': 48, 'surround': 49, 'reverber': 50, 'tap': 51, 'cane': 52, 'hors': 53, 'hoofbeat': 54, 'worth': 55, 'famou': 56, 'crimin': 57, 'possibl': 58, 'inspir': 59, 'sherlock': 60, 'holm': 61, 'nemesi': 62, 'professor': 63, 'moriarti': 64, 'compris': 65, 'kensal': 66, 'green': 67, 'west': 68, 'norwood': 69, 'abney': 70, 'nunhead': 71, 'brompton': 72, '

### 2.1.1 Create your index!

We have already created the vocabulary file as a dictionary in the last point of the exercise, now we need to invert the index and match the words in the dict with the words in the place's description.

In [ ]:
import json
path='/content/drive/MyDrive/ADM_HW3'
with open (path+'vocabulary.json', 'w',encoding = 'utf-8') as j:
    j.write(json.dumps(vocabulary))

Here we create a list of all the cleaned description and we put it into the tsv file as a new column. That will help us when we will have to execute the query and return the place title and the url in the output.

In [ ]:
list_clean_desc=[]

for i in visual.placeDesc:
  clean_desc=clean(str(i))
  list_clean_desc.append(clean_desc)

In [ ]:
visual_copy=visual.copy()
visual_copy["list_clean_desc"]=list_clean_desc

visual_copy.head(3)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL,list_clean_desc
0,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2625.0,8208.0,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,['anthonyhogg'],"August 9, 2014","[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...,"[open, 1839, highgat, one, london, infam, ceme..."
2,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834.0,8606.0,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['rebekah-otto', 'annetta-black']","May 8, 2010",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...,"[first, new, york, citi, subway, built, oper, ..."
3,Leadenhall Market,"['markets', 'film-locations', 'film']",3142.0,7595.0,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['meghanneal', 'gavin-292130']","August 1, 2016","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...,"[ornat, 19thcenturi, paint, roof, cobbl, floor..."


In [ ]:
#RI-RUNNARE

inverted_idx = {}

for word, item_id in vocabulary.items():
    inverted_idx[item_id] = list(visual_copy[visual_copy.list_clean_desc.apply(lambda row: word in row)].index)
  

In [ ]:
# NON runnare
with open('/content/drive/MyDrive/ADM_HW3/invertedidx.pkl', 'wb') as f:
  pickle.dump(inverted_idx, f)

In [ ]:
# NON runnare
del inverted_idx

In [ ]:
with open('/content/drive/MyDrive/ADM_HW3/invertedidx.pkl', 'rb') as f:
  inverted_idx = pickle.load(f)

### 2.1.2 Execute the query

We are now defining a function that clean the query as we have cleaned the descriptions and then saves the index of the document that contains all the words of the given query in a list. We will then use this list to retrieve the final output of the search engine.

In [ ]:

def output_id(query):
  c=query.split(' ')

  new_parole=[]
  for i in c:
    p = clean(i)
    new_parole.append(p[0])

  documents=[]
  for parola in new_parole:
    index_word= vocabulary[parola]
    docs= inverted_idx[index_word]
    documents.append(docs)

  fla_documents=[]
  for elem in documents:
    for num in elem:
      fla_documents.append(num)

  w=len(new_parole) #2
  output_places=[]

  for i in fla_documents:
    c = fla_documents.count(i)
    if c == w and i not in output_places:
      output_places.append(i)
  
  return output_places

Let's try the function and see if it works

In [ ]:
query = 'American Museum'
print(output_id(query))
print(len(output_id(query)))

[60, 141, 174, 225, 321, 329, 358, 361, 417, 692, 785, 789, 893, 894, 961, 972, 974, 1007, 1039, 1052, 1075, 1091, 1101, 1110, 1123, 1126, 1163, 1199, 1204, 1234, 1249, 1261, 1331, 1355, 1365, 1387, 1401, 1414, 1418, 1485, 1541, 1544, 1554, 1557, 1582, 1588, 1626, 1701, 1738, 1766, 1777, 1802, 1804, 1805, 1858, 1859, 1888, 1902, 1970, 1996, 1999, 2058, 2072, 2078, 2088, 2110, 2138, 2167, 2191, 2221, 2223, 2236, 2276, 2280, 2286, 2364, 2418, 2468, 2559, 2561, 2579, 2596, 2653, 2662, 2665, 2670, 2726, 2737, 2744, 2795, 2826, 2956, 3009, 3053, 3081, 3089, 3168, 3170, 3217, 3263, 3274, 3354, 3369, 3375, 3397, 3410, 3417, 3418, 3420, 3517, 3615, 3687, 3698, 3713, 3722, 3757, 3775, 3795, 3811, 3827, 3832, 3896, 3969, 4011, 4044, 4057, 4098, 4099, 4104, 4269, 4295, 4297, 4320, 4376, 4393, 4396, 4405, 4416, 4423, 4430, 4449, 4459, 4473, 4490, 4500, 4512, 4533, 4542, 4546, 4548, 4554, 4563, 4634, 4676, 4691, 4735, 4766, 4778, 4784, 4787, 4864, 4933, 4948, 4952, 4965, 4987, 4995, 5066, 5070, 507

In [ ]:
print('The id of the places corresponding to the query are '+str(output_places))

The places corresponding to the query are [4272]


Now that we have the place_id that we want to print, let's find in the tsv file all the information we want to give in output.

In [ ]:
def SearchEngine(query):
  pl = output_id(query)
  col = ['placeName', 'placeDesc', 'placeURL']
  df = pd.DataFrame(columns=col)
  df.placeName = [visual.placeName[i] for i in pl]
  df.placeDesc = [visual.placeDesc[i] for i in pl]
  df.placeURL = [visual.placeURL[i] for i in pl]
  return df

Give it a try!

In [ ]:
frs = 'American Museum'
SearchEngine(frs)

,placeName,placeDesc,placeURL
0,The Witch House of Salem,The Salem witchcraft trials took place between...,http://www.atlasobscura.com/places/witch-house...
1,Museum of the Weird,The dime or dime store museum is by all accoun...,http://www.atlasobscura.com/places/museum-weird
2,National Atomic Testing Museum,Las Vegas is an oasis in the desert where no o...,http://www.atlasobscura.com/places/national-at...
3,Canyons of the Ancients,Ripe for quiet reflection and simply awe-inspi...,http://www.atlasobscura.com/places/canyons-of-...
4,The Natural Bridge,Often cited as having a place among the great ...,http://www.atlasobscura.com/places/the-natural...
...,...,...,...
238,De Vargas Street House,"Located in Santa Fe, New Mexico, the De Vargas...",http://www.atlasobscura.com/places/de-vargas-s...
239,The Liberty Bell Hiding Place,"In September of 1777, the British defeated Geo...",http://www.atlasobscura.com/places/liberty-bel...
240,Friendship Park,"On most days, San Diego’s Friendship Park is a...",http://www.atlasobscura.com/places/friendship-...
241,American Computer Museum,"“To collect, preserve, interpret, and display ...",http://www.atlasobscura.com/places/american-co...


## 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index 2.0

In [ ]:
#tf-ij = # occorrenze del termine i nel documento j / lunghezza documento (num parole)

#idf-i = log10 ( #documenti in totale (7200) / #documenti che contengono il termine i (len(inverted_idx[parola])))

In [ ]:
# 1 : 2, 5 ,7   ----> 1 : {2 : 0,6} {5 : 0,012} {7 : 0,09}

In [ ]:
print(len(list_clean_desc))

7200


In [ ]:
# NON runnare
from nltk.corpus.reader.ycoe import defaultdict
D=7200
inverted_idx_2 = defaultdict(dict)
for k, i in tqdm(vocabulary.items()):
  idf_i = np.log(D/len(inverted_idx[i]))
  for j in range(D):
    if k in list_clean_desc[j]:
      lung = len(list_clean_desc[j])
      freq = list_clean_desc[j].count(k)
      tf_ij = freq/lung
      tf_idf= tf_ij * idf_i
      inverted_idx_2[i].update({ j : tf_idf})

100%|██████████| 48986/48986 [25:12<00:00, 32.39it/s]


In [ ]:
# NON runnare
with open('/content/drive/MyDrive/ADM_HW3/invertedidx2.pkl', 'wb') as f:
  pickle.dump(inverted_idx_2, f)

del inverted_idx_2

In [ ]:
with open('/content/drive/MyDrive/ADM_HW3/invertedidx2.pkl', 'rb') as f:
  inverted_idx_2 = pickle.load(f)

In [ ]:
visual.shape[0]

7200

### 2.2.2 Execute the query

This function is an adaptation of the previous one. In this new version we use the inverted index 2.0 with the tfidf associated to each document for each word, and the output will be a list of index that corresponds to the document that compete for the last output.

In [ ]:
def output_id2(query):
  c=query.split(' ')

  new_parole=[]
  for i in c:
    p = clean(i)
    new_parole.append(p[0])
  
  #add also the tfidf of the words in the query
  documents={}
  tf_idf_q={}
  N=visual.shape[0]
  for parola in new_parole:
    index_word= vocabulary[parola]
    docs= inverted_idx_2[index_word]
    documents[index_word]=docs
    tf_ij_q = new_parole.count(parola)
    idf_i_q = len(inverted_idx_2[index_word])
    tf_idf_q[index_word] = tf_ij_q * np.log(N/idf_i_q)

  fla_documents=list(documents.values())
  all_documents=[list(fla_documents[x].keys()) for x in range(len(fla_documents))]
  
  final_documents=[]
  for elem in all_documents:
    for num in elem:
      final_documents.append(num)

  w=len(new_parole) #2
  output_places2=[]

  for i in final_documents:
    c = final_documents.count(i)
    if c == w and i not in output_places2:
      output_places2.append(i)
  
  return output_places2, tf_idf_q

In [ ]:
frase = 'American museum'

output_id2(frase)

In [ ]:
def getNorm(id_desc, doc):
  word=doc[id_desc]
  total_tfidf=[]
  for par in word:
    idx = vocabulary[par]
    position = inverted_idx_2[idx]
    tfidf_par_doc = position[id_desc]
    total_tfidf.append(tfidf_par_doc)
  
  desc_norm = np.linalg.norm(total_tfidf)

  return desc_norm

Now we apply this function to all the description we have cleaned and stored in a list, and then create a new list which contains the ordered norm of each description.

In [ ]:
norme = []
for descrizione in range(len(list_clean_desc)):
  norme.append(getNorm(descrizione, list_clean_desc))

An example to see if it works.

In [ ]:
getNorm(69,list_clean_desc)

0.7665270630310047

In [ ]:
def Classifica(query, k):
  candidati, tfidfquery = output_id2(query)
  classifica = []
  heapq.heapify(classifica)
  #per ogni documento candidato
  for doc in candidati:
    prodotti=[]
    #per ogni parola della query
    for parola in list(tfidfquery.keys()):
      tfidf_p = inverted_idx_2[parola]
      tfidf_pd = tfidf_p[doc]
      prodotti.append(tfidf_pd * tfidfquery[parola])
    #cosine similarity
    somma = sum(prodotti)
    cosine = somma/norme[doc]
    heapq.heappush(classifica, (cosine, doc))
    if len(classifica)==(k+1):
      heapq.heappop(classifica)

  return classifica

In [ ]:
frase = 'american museum'

Classifica(frase, 5)

[(0.3377358950850909, 5269),
 (0.37954223152103844, 2664),
 (0.36005560482222176, 6209),
 (0.39238307158521707, 4472),
 (0.3964233975611651, 4994)]

In [90]:
def searchEngine(query, k):
  out=Classifica(query, k)

  col = ['placeName', 'placeDesc', 'placeURL', 'cosineSimilarity']
  df = pd.DataFrame(columns=col)
  
  i=0
  for cos, ind in out:
    df.at[i, 'placeName'] = visual.placeName[ind]
    df.at[i, 'placeDesc'] = visual.placeDesc[ind]
    df.at[i, 'placeURL'] = visual.placeURL[ind]
    df.at[i, 'cosineSimilarity'] = cos
    i+=1

  finito = df.sort_values(by=['cosineSimilarity'], ascending=False)

  return finito

In [ ]:
visual.placeName[5000]

'Rummu Underwater Prison'

In [91]:
frase = 'american museum'

searchEngine(frase, 10)

,placeName,placeDesc,placeURL,cosineSimilarity
9,The Czech's Capuchin Crypt,The rosy pink facade of Brno’s Capuchin Church...,http://www.atlasobscura.com/places/czechs-capu...,0.396423
8,Hill of Witches,"Just off the main street in Juodkrante, Lithua...",http://www.atlasobscura.com/places/hill-of-wit...,0.392383
7,Cahokia Mounds,The largest pre-Columbian settlement in the Ne...,http://www.atlasobscura.com/places/cahokia-mounds,0.379542
3,Sea of Stars,The “Sea of Stars” is a beautiful phenomenon t...,http://www.atlasobscura.com/places/sea-of-stars,0.360056
4,Slope Point,"Slope Point, the most southerly spot on New Ze...",http://www.atlasobscura.com/places/slope-point,0.337736
1,Evergreen Memorial Park & Crematory,"Established in 1877, Evergreen Memorial Park a...",http://www.atlasobscura.com/places/evergreen-m...,0.32864
5,Crowley Mausoleum,"For over a century, the Crowley family owned t...",http://www.atlasobscura.com/places/crowley-mau...,0.308192
6,Ladder Creek Falls and Gardens,Ladder Creek Falls and Gardens in North Cascad...,http://www.atlasobscura.com/places/ladder-cree...,0.299304
2,The Miller Building,There’s no way to know how many reporters were...,http://www.atlasobscura.com/places/the-miller-...,0.281186
0,Thain Family Forest,This small forest in the Bronx could be the mo...,http://www.atlasobscura.com/places/thain-famil...,0.278352


## 3. Define a new score!

## 4. Visualizing the most relevant places

In [ ]:
import plotly.express as px

In [ ]:
dis = px.scatter_mapbox( AGGIUNGERE, lat='plceAlt', lon='placeLong', hover_name='placeName', 
                        hover_data=['numPeopleVisited', 'placeAddress'], color='numPeopleVisited', 
                        color_continuois_scale='algae', size='numPeopleWant', zoom=2, height=300)

dis.update_layout(mapbox_style='open-street-map')
dis.update_layout(margin={'r': 0 , 't' : 0 , 'l' : 0 , 'b' : 0})

dif.show()

## 5. BONUS: More complex search engine

## 6. Command line question

## 7. Theoretical question